In [1]:
import gymnasium as gym
import ale_py  # Ensure Atari environments work
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
import collections
import random
from collections import deque
import torch.nn.functional as F
import cv2
from tqdm import tqdm
import wandb
from functools import partial
import os

from utils import get_env, record_video_oc, get_obj_classes

In [2]:
class DQN_OC(nn.Module):
    def __init__(self, num_classes, hidden_dim, action_dim):
        super(DQN_OC, self).__init__()

        self.class_embs = nn.Embedding(num_embeddings=num_classes, embedding_dim=hidden_dim, padding_idx=0)
        self.time_embs = nn.Embedding(num_embeddings=4, embedding_dim=hidden_dim)
        self.xywh_proj = nn.Linear(4, hidden_dim)

        self.encoder = nn.GRU(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=3,
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, action_dim)  # Output Q-values for each action
        )

    def forward(self, x):
        time_emb = self.time_embs(x[:, :, 0].long())
        class_emb = self.class_embs(x[:, :, 1].long())
        xywh_emb = self.xywh_proj(x[:, :, 2:])
        obj_emb = time_emb + class_emb + xywh_emb
        x, _ = self.encoder(obj_emb)
        x, _ = torch.max(x, dim=1)
        x = self.fc_layers(x)
        return x

In [3]:
def select_action(env, model, state_objs, epsilon):
    if random.random() < epsilon:
        return env.action_space.sample()  # Random action (exploration)

    with torch.no_grad():
        # Add batch dimension
        state_objs = state_objs.to_tensor()
        state_objs = state_objs.unsqueeze(0).to(device)
        return model(state_objs).squeeze().argmax().item()

def train(model, target_model, buffer, optimizer, batch_size, gamma):
    # print('train buffer.size():', buffer.size())
    # print('batch_size:', batch_size)

    if buffer.size() < batch_size:
        return 0
    
    # Sample batch from experience replay
    state_objs, actions, rewards, next_state_objs, dones = buffer.sample(batch_size)

    state_objs = state_objs.to(device)
    actions = actions.to(device)
    rewards = rewards.to(device)
    dones = dones.to(device)
    next_state_objs = next_state_objs.to(device)

    # Compute Q-values for current states
    q = model(state_objs)
    # print('q.shape:', q.shape)
    q_values = q.gather(1, actions.unsqueeze(1)).squeeze(1)  # Select Q-values of taken actions

    # Compute next Q-values from the target network
    next_q_values = target_model(next_state_objs).max(1)[0].detach()  # Max Q-value of next state

    dones = dones.to(torch.bool)
    # Zero next_q_values for terminal states
    next_q_values[dones] = 0.0

    # Compute target Q-values
    scaled_rewards = 0.01 * rewards
    target_q_values = scaled_rewards + gamma * next_q_values

    dq_loss = F.mse_loss(q_values, target_q_values.detach())

    # Backpropagation
    optimizer.zero_grad()
    dq_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
    optimizer.step()
    return dq_loss.item()


In [4]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def push(self, state_objs, action, reward, next_state_objs, done):
        self.buffer.append((
            state_objs,
            action,
            int(reward),
            next_state_objs,
            bool(done)
        ))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state_objs, action, reward, next_state_objs, done = zip(*batch)

        return (
            pad_sequence((objs.to_tensor() for objs in state_objs), batch_first=True, padding_value=0.0),
            torch.LongTensor(action),
            torch.FloatTensor(reward),
            pad_sequence((objs.to_tensor() for objs in next_state_objs), batch_first=True, padding_value=0.0),
            torch.FloatTensor(done)
        )

    def size(self):
        return len(self.buffer)

In [5]:
# Create the Atari environment
knn=3
env = get_env(process=False, oc=True, knn=knn)

# Check Action / State space
obs, info = env.reset()

action_dim = env.action_space.n

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dqn = DQN_OC(num_classes=len(get_obj_classes())+1, hidden_dim=300, action_dim=action_dim).to(device)
target_dqn = DQN_OC(num_classes=len(get_obj_classes())+1, hidden_dim=300, action_dim=action_dim).to(device)
target_dqn.load_state_dict(dqn.state_dict())

lr = 0.0001
weight_decay = 1e-5
replay_buffer_size = 10000
optimizer = optim.AdamW(dqn.parameters(), lr=lr, weight_decay=weight_decay)
replay_buffer = ReplayBuffer(replay_buffer_size)

num_train_iterations = 1000000
batch_size = 32
gamma = 0.99
epsilon = 1.0
epsilon_min = 0.05
epsilon_decay = 0.999885
target_update_freq = 10000
rewards_list = []

project_name = 'dqn_oc_knn'

wandb.require("core")
wandb.login()
wandb.init(
      # Set the project where this run will be logged
      project="frogger",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=project_name,
      # Track hyperparameters and run metadata
      config={
      "lr": lr,
      "weight_decay": weight_decay,
      "batch_size": batch_size,
      "gamma": gamma,
      "epsilon": epsilon,
      "epsilon_min": epsilon_min,
      "epsilon_decay": epsilon_decay,
      "replay_buffer_size": replay_buffer_size,
      "variant": project_name,
      "num_train_iterations": num_train_iterations,
      "target_update_freq": target_update_freq,
      })

A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]
wandb: WARNING `wandb.require('core')` is redundant as it is now the default behavior.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kevinxli to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [6]:
os.makedirs(f'{project_name}/train', exist_ok=True)

optimizer = optim.AdamW(dqn.parameters(), lr=lr, weight_decay=weight_decay)
state, info = env.reset()
total_loss = 0
total_reward = 0

for iteration in tqdm(range(num_train_iterations+1)):
    action = select_action(env, dqn, state, epsilon)
    next_state, reward, terminated, truncated, info = env.step(action)
    total_reward += reward

    replay_buffer.push(state, action, reward, next_state, terminated)

    if terminated or truncated:
        state, info = env.reset()
        wandb.log({'train/loss': total_loss})
        total_loss = 0
        total_reward = 0
    else:
        state = next_state

    loss = train(dqn, target_dqn, replay_buffer, optimizer, batch_size, gamma)
    total_loss += loss
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    rewards_list.append(total_reward)

    if iteration % target_update_freq == 0:
        target_dqn.load_state_dict(dqn.state_dict())
        torch.save(dqn.state_dict(), f"{project_name}/train/frogger_dqn_iter_{iteration}.pth")
        if iteration > 0:
            reward, length, time = record_video_oc(select_action=partial(select_action, model=dqn, epsilon=0), video_folder=f"{project_name}/videos", video_name=f"train_iter_{iteration}", knn=knn)
            wandb.log({'train/reward': reward, 'train/length': length, 'train/time': time})


  0%|          | 0/1000001 [00:00<?, ?it/s]

  1%|          | 9993/1000001 [02:00<3:16:26, 83.99it/s] 

iteration 0 action 0
iteration 1 action 0
iteration 2 action 0
iteration 3 action 0
iteration 4 action 0
iteration 5 action 0
iteration 6 action 0
iteration 7 action 0
iteration 8 action 0
iteration 9 action 0
iteration 10 action 0
iteration 11 action 0
iteration 12 action 0
iteration 13 action 0
iteration 14 action 0
iteration 15 action 0
iteration 16 action 0
iteration 17 action 0
iteration 18 action 0
iteration 19 action 0
iteration 20 action 0
iteration 21 action 0
iteration 22 action 2
iteration 23 action 2
iteration 24 action 2
iteration 25 action 2
iteration 26 action 2
iteration 27 action 2
iteration 28 action 2
iteration 29 action 2
iteration 30 action 2
iteration 31 action 2
iteration 32 action 2
iteration 33 action 2
iteration 34 action 2
iteration 35 action 2
iteration 36 action 2
iteration 37 action 2
iteration 38 action 2
iteration 39 action 2
iteration 40 action 2
iteration 41 action 2
iteration 42 action 2
iteration 43 action 2
iteration 44 action 2
iteration 45 action 

  1%|          | 9993/1000001 [02:20<3:16:26, 83.99it/s]

iteration 1063 action 2
iteration 1064 action 2
iteration 1065 action 2
iteration 1066 action 2
iteration 1067 action 2
iteration 1068 action 2
iteration 1069 action 2
iteration 1070 action 2
iteration 1071 action 2
iteration 1072 action 2
iteration 1073 action 2
iteration 1074 action 2
iteration 1075 action 2
iteration 1076 action 2
iteration 1077 action 2
iteration 1078 action 2
iteration 1079 action 2
iteration 1080 action 2
iteration 1081 action 2
iteration 1082 action 2
iteration 1083 action 2
iteration 1084 action 2
iteration 1085 action 2
iteration 1086 action 2
iteration 1087 action 2
iteration 1088 action 2
iteration 1089 action 2
iteration 1090 action 2
iteration 1091 action 2
iteration 1092 action 2
iteration 1093 action 2
iteration 1094 action 2
iteration 1095 action 2
iteration 1096 action 2
iteration 1097 action 2
iteration 1098 action 2
iteration 1099 action 2
iteration 1100 action 2
iteration 1101 action 2
iteration 1102 action 2
iteration 1103 action 2
iteration 1104 a

...

iteration 0 action 3
iteration 1 action 3
iteration 2 action 3
iteration 3 action 3
iteration 4 action 3
iteration 5 action 3
iteration 6 action 3
iteration 7 action 3
iteration 8 action 3
iteration 9 action 3
iteration 10 action 3
iteration 11 action 3
iteration 12 action 3
iteration 13 action 3
iteration 14 action 3
iteration 15 action 3
iteration 16 action 3
iteration 17 action 3
iteration 18 action 3
iteration 19 action 3
iteration 20 action 3
iteration 21 action 3
iteration 22 action 3
iteration 23 action 3
iteration 24 action 3
iteration 25 action 3
iteration 26 action 3
iteration 27 action 3
iteration 28 action 3
iteration 29 action 3
iteration 30 action 3
iteration 31 action 3
iteration 32 action 3
iteration 33 action 3
iteration 34 action 3
iteration 35 action 3
iteration 36 action 3
iteration 37 action 3
iteration 38 action 3
iteration 39 action 3
iteration 40 action 3
iteration 41 action 3
iteration 42 action 3
iteration 43 action 3
iteration 44 action 3
iteration 45 action 

 13%|█▎        | 129994/1000001 [41:30<3:29:26, 69.23it/s]

iteration 719 action 3
iteration 720 action 3
iteration 721 action 3
iteration 722 action 3
iteration 723 action 3
iteration 724 action 3
iteration 725 action 3
iteration 726 action 3
iteration 727 action 3
iteration 728 action 3
iteration 729 action 3
iteration 730 action 3
iteration 731 action 3
iteration 732 action 3
iteration 733 action 3
iteration 734 action 3
iteration 735 action 3
iteration 736 action 3
iteration 737 action 3
iteration 738 action 3
iteration 739 action 3
iteration 740 action 3
iteration 741 action 3
iteration 742 action 3
iteration 743 action 3
iteration 744 action 3
iteration 745 action 3
iteration 746 action 3
iteration 747 action 3
iteration 748 action 3
iteration 749 action 3
iteration 750 action 3
iteration 751 action 3
iteration 752 action 3
iteration 753 action 3
iteration 754 action 3
iteration 755 action 3
iteration 756 action 3
iteration 757 action 3
iteration 758 action 3
iteration 759 action 3
iteration 760 action 3
iteration 761 action 3
iteration 7

 13%|█▎        | 130000/1000001 [41:47<4:39:39, 51.85it/s]


KeyboardInterrupt: 